Создание базы данных SQLite и таблицы


In [1]:
import sqlite3
import pandas as pd

# Создаем подключение к базе данных (файл базы данных будет создан)
conn = sqlite3.connect('sales_data.db')

# Создаем курсор для выполнения SQL-запросов
cursor = conn.cursor()

# Создаем таблицу для хранения данных о продажах
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales (
    OrderID INTEGER PRIMARY KEY,
    Product TEXT,
    Quantity INTEGER,
    Price REAL,
    Date TEXT
)
''')

# Сохраняем изменения и закрываем курсор
conn.commit()

Теперь мы загрузим данные о продажах в нашу таблицу.

In [2]:
# Пример данных о продажах
data = {
    'OrderID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Product': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C'],
    'Quantity': [10, 20, 15, 5, 10, 25, 10, 30, 5, 20],
    'Price': [100, 200, 150, 300, 250, 100, 300, 200, 150, 300],
    'Date': pd.to_datetime([
        '2023-01-01', '2023-01-02', '2023-01-02',
        '2023-01-03', '2023-01-03', '2023-01-04',
        '2023-01-04', '2023-01-05', '2023-01-05', '2023-01-06'
    ]).astype(str)  # Преобразуем даты в строковый формат
}

# Вставляем данные в таблицу
for i in range(len(data['OrderID'])):
    cursor.execute('''
    INSERT INTO sales (OrderID, Product, Quantity, Price, Date) VALUES (?, ?, ?, ?, ?)
    ''', (data['OrderID'][i], data['Product'][i], data['Quantity'][i], data['Price'][i], data['Date'][i]))

# Сохраняем изменения и закрываем курсор
conn.commit()

Теперь создадим OLAP-куб с помощью SQL-запроса, который будет агрегировать данные по продуктам и датам.

In [3]:
# SQL-запрос для создания OLAP-куба
query = '''
SELECT
    Date,
    Product,
    SUM(Quantity) AS Total_Quantity,
    SUM(Price * Quantity) AS Total_Sales
FROM
    sales
GROUP BY
    Date, Product
ORDER BY
    Date, Product
'''

# Выполняем запрос и загружаем данные в DataFrame
olap_cube = pd.read_sql_query(query, conn)

print("OLAP-куб:")
print(olap_cube)


OLAP-куб:
         Date Product  Total_Quantity  Total_Sales
0  2023-01-01       A              10       1000.0
1  2023-01-02       A              15       2250.0
2  2023-01-02       B              20       4000.0
3  2023-01-03       B              10       2500.0
4  2023-01-03       C               5       1500.0
5  2023-01-04       A              25       2500.0
6  2023-01-04       C              10       3000.0
7  2023-01-05       A               5        750.0
8  2023-01-05       B              30       6000.0
9  2023-01-06       C              20       6000.0


Теперь мы можем выгрузить данные из OLAP-куба в pandas и провести дальнейший анализ.

In [5]:
# Выгрузка данных по продукту A
product_a_data = olap_cube[olap_cube['Product'] == 'A']
print("\nДанные по продукту A:")
print(product_a_data)

# Общая сумма продаж по датам
total_sales_per_date = olap_cube.groupby('Date')['Total_Sales'].sum()
print("\nОбщая сумма продаж по датам:")
print(total_sales_per_date)


Данные по продукту A:
         Date Product  Total_Quantity  Total_Sales
0  2023-01-01       A              10       1000.0
1  2023-01-02       A              15       2250.0
5  2023-01-04       A              25       2500.0
7  2023-01-05       A               5        750.0

Общая сумма продаж по датам:
Date
2023-01-01    1000.0
2023-01-02    6250.0
2023-01-03    4000.0
2023-01-04    5500.0
2023-01-05    6750.0
2023-01-06    6000.0
Name: Total_Sales, dtype: float64
